In [2]:
#import snowflake.connector
#from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np

## Funciones: 

In [3]:
# FUNCIONES
def Columnames(database, namebase):
    """ * Eliminar las columnas y filas sin datos
        * Agregar '_' entre los espacios 
        * Agregar nombre """
    deletecol = 0
    a = 0
    a = database.columns
    a = list(a)
    deletecol = [lista for lista in a if "Unnamed" in lista]
    if(deletecol != 0):
        database = database.drop(columns = deletecol)
    
    #database = database.loc[0:9,:]
    database= database.dropna(how='all')
    new_names = []
    for i in database.columns.values:
        result = i.replace(' ','_')
        #result = result + namebase
        new_names.append(result)
    database.columns = new_names
    return database

In [4]:
# Bases de datos
df = pd.read_excel("DELIV.xlsx", sheet_name='Sheet1')
df2 = pd.read_excel("DSERV.xlsx", sheet_name='Sheet1')
df3 = pd.read_excel("CALL.xlsx", sheet_name='Sheet1')
df4 = pd.read_excel("EMP.xlsx", sheet_name='Sheet1')

In [5]:
DELIV = df
DSERV = df2
EMP = df3
CALL = df4

In [6]:
DELIV = Columnames(DELIV, "_DELIV")
DSERV = Columnames(DSERV, "_DSERV")
EMP = Columnames(EMP, "_EMP")
CALL = Columnames(CALL, "_CALL")

## Procesamiento de los datos de las bases

# Z1045_DELYIVERY

In [8]:
DELIV["Fecha_inicio_real"] = DELIV["Fecha_inicio_real"].astype(str)
DELIV["In-Plant_Hour_of_Delivery"] = DELIV["In-Plant_Hour_of_Delivery"].astype(str)
DELIV["Fecha_de_entrega"] = DELIV["Fecha_de_entrega"].astype(str)
DELIV["In_Site_Date"] = DELIV["In_Site_Date"].astype(str)
DELIV["Unld_Initial_Date"] = DELIV["Unld_Initial_Date"].astype(str)
DELIV["Unld_Final_Date"] = DELIV["Unld_Final_Date"].astype(str)
DELIV["Date_of_Assigned_Delivery"] = DELIV["Date_of_Assigned_Delivery"].astype(str)
DELIV["Fecha_de_documento"] = DELIV["Fecha_de_documento"].astype(str)
DELIV["Fecha_de_entrega.1"] = DELIV["Fecha_de_entrega.1"].astype(str)
DELIV["Fecha_de_carga"] = DELIV["Fecha_de_carga"].astype(str)
DELIV["Fecha_real_de_fin_de_la_orden"] = DELIV["Fecha_real_de_fin_de_la_orden"].astype(str)
DELIV["Fecha_TJST"] = DELIV["Fecha_TJST"].astype(str)
DELIV["Fecha_TOPL"] = DELIV["Fecha_TOPL"].astype(str)

In [9]:
DELIV = DELIV.rename({'In-Plant_Hour_of_Delivery': 'In_Plant_Hour_of_Delivery'}, axis=1)

In [10]:
DELIV["Fecha_TJST"] = DELIV.Fecha_TJST.astype(str).str.split(' ', expand=True)[0]
DELIV["In_Site_Date"] = DELIV.In_Site_Date.astype(str).str.split(' ', expand=True)[0]
DELIV["Unld_Initial_Date"] = DELIV.Unld_Initial_Date.astype(str).str.split(' ', expand=True)[0]
DELIV["Unld_Final_Date"] = DELIV.Unld_Final_Date.astype(str).str.split(' ', expand=True)[0]
DELIV["Fecha_TOPL"] = DELIV.Fecha_TOPL.astype(str).str.split(' ', expand=True)[0]
DELIV["In_Plant_Hour_of_Delivery"] = DELIV.In_Plant_Hour_of_Delivery.astype(str).str.split(' ', expand=True)[0]
DELIV["Date_of_Assigned_Delivery"] = DELIV.Date_of_Assigned_Delivery.astype(str).str.split(' ', expand=True)[0]
DELIV["Fecha_de_carga"] = DELIV.Fecha_de_carga.astype(str).str.split(' ', expand=True)[0]
DELIV["Fecha_de_entrega.1"] = DELIV["Fecha_de_entrega.1"].astype(str).str.split(' ', expand=True)[0]

In [11]:
DELIV["Fecha_TJST"] = DELIV["Fecha_TJST"].astype(str) + " " + DELIV["Hora_TJST"].astype(str)
DELIV["In_Site_Date"] = DELIV["In_Site_Date"].astype(str) + " " + DELIV["In_Site_Hr"].astype(str)
DELIV["Unld_Initial_Date"] = DELIV["Unld_Initial_Date"].astype(str) + " " + DELIV["Unld_Initial_Hr"].astype(str)
DELIV["Unld_Final_Date"] = DELIV["Unld_Final_Date"].astype(str) + " " + DELIV["Unld_Final_Hr"].astype(str)
DELIV["Fecha_TOPL"] = DELIV["Fecha_TOPL"].astype(str) + " " + DELIV["Hora_TOPL"].astype(str)
DELIV["In_Plant_Hour_of_Delivery"] = DELIV["In_Plant_Hour_of_Delivery"].astype(str) + " " + DELIV["In-Plant_Hour_of_Delivery.1"].astype(str)
DELIV["Date_of_Assigned_Delivery"] = DELIV["Date_of_Assigned_Delivery"].astype(str) + " " + DELIV["Hour_of_Assigned_Delivery"].astype(str)
DELIV["Fecha_de_carga"] = DELIV["Fecha_de_carga"].astype(str) + " " + DELIV["Hora_de_carga"].astype(str)
DELIV["Fecha_de_entrega.1"] = DELIV["Fecha_de_entrega.1"].astype(str) + " " + DELIV["Hora_de_entrega"].astype(str)

In [12]:
DELIV["Fecha_TJST"] = pd.to_datetime(DELIV["Fecha_TJST"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["In_Site_Date"] = pd.to_datetime(DELIV["In_Site_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Unld_Initial_Date"] = pd.to_datetime(DELIV["Unld_Initial_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Unld_Final_Date"] = pd.to_datetime(DELIV["Unld_Final_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Fecha_TOPL"] = pd.to_datetime(DELIV["Fecha_TOPL"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["In_Plant_Hour_of_Delivery"] = pd.to_datetime(DELIV["In_Plant_Hour_of_Delivery"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Date_of_Assigned_Delivery"] = pd.to_datetime(DELIV["Date_of_Assigned_Delivery"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Fecha_de_carga"] = pd.to_datetime(DELIV["Fecha_de_carga"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DELIV["Fecha_de_entrega.1"] = pd.to_datetime(DELIV["Fecha_de_entrega.1"], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [13]:
DELIV = DELIV.drop(["Hora_TJST","In_Site_Hr","Unld_Initial_Hr","Unld_Final_Hr","Hora_TOPL","In-Plant_Hour_of_Delivery.1","Hour_of_Assigned_Delivery","Hora_de_carga","Hora_de_entrega"], axis=1)

In [14]:
# Filtrar los datos unicos para realizar las comparaciones 
Unicos = DELIV[DELIV.Entrega.duplicated() == True]
Unicos = Unicos.Entrega.unique()
Unicos = pd.DataFrame(Unicos)
Unicos = Unicos.rename({0: 'Entrega'}, axis=1)
Duplicados = Unicos.merge(DELIV, on='Entrega', how='left')

In [15]:
Duplicados["Ponderacion1"] = np.where(Duplicados["Fecha_TJST"] >= Duplicados["In_Site_Date"], 0, 1)
Duplicados["Ponderacion2"] = np.where(Duplicados["In_Site_Date"] >= Duplicados["Unld_Initial_Date"], 0, 1)
Duplicados["Ponderacion3"] = np.where(Duplicados["Unld_Initial_Date"] >= Duplicados["Unld_Final_Date"], 0, 1)
Duplicados["Ponderacion4"] = np.where(Duplicados["Unld_Final_Date"] >= Duplicados["Fecha_TOPL"], 0, 1)
Duplicados["Ponderacion5"] = np.where(Duplicados["Fecha_TOPL"] >= Duplicados["In_Plant_Hour_of_Delivery"], 0, 1)
Duplicados["Ponderacion6"] = np.where(Duplicados["In_Plant_Hour_of_Delivery"] >= Duplicados["Date_of_Assigned_Delivery"], 0, 1)
Duplicados["Ponderacion7"] = np.where(Duplicados["Date_of_Assigned_Delivery"] >= Duplicados["Fecha_de_carga"], 0, 1)
Duplicados["Ponderacion8"] = np.where(Duplicados["Fecha_de_carga"] >= Duplicados["Fecha_de_entrega.1"], 0, 1)

In [16]:
Duplicados["Ponderacion"] = Duplicados["Ponderacion1"] + Duplicados["Ponderacion2"] + Duplicados["Ponderacion3"] + Duplicados["Ponderacion4"] + Duplicados["Ponderacion5"] + Duplicados["Ponderacion6"] + Duplicados["Ponderacion7"] + Duplicados["Ponderacion8"]
Duplicados = Duplicados.drop(["Ponderacion1","Ponderacion2","Ponderacion3","Ponderacion4","Ponderacion5","Ponderacion6","Ponderacion7","Ponderacion8"], axis=1)

In [17]:
Duplicados = Duplicados.sort_values('Ponderacion',ascending=False)
Duplicados = Duplicados[Duplicados.Entrega.duplicated() == False]
Duplicados = Duplicados.drop(["Ponderacion"], axis=1)

In [18]:
Data = DELIV.merge(Unicos, on='Entrega', how='left',indicator=True)
Data = Data[Data['_merge'] == "left_only"]
temp = Data["Entrega"]
Data = Data.drop(["_merge","Entrega"], axis=1)
Data.insert(0, 'Entrega', temp)

In [19]:
Full = pd.concat([Data, Duplicados])

In [20]:
Full

,Entrega,Centro,Conductor,Cantidad_entrega,Fecha_inicio_real,Hora_inicio_real,Nombre_de_la_Obra,Documento_de_Venta,Placa_Vehículo_1,Carga,...,Tiempo_de_trayecto,T._Validación_mezcla_(mins),Distancia,Ciclo_Real,Ciclo_Teórico,Nombre_del_Frente,Cycle_Time,Fecha_TJST,Fecha_TOPL,Motivo_de_Pedido
0,223802261,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,...,00:20:00,20,5.0,01:41:00,02:02:00,ANTARA FASE II,02:01:24,2022-11-30 15:31:40,2022-11-30 15:57:14,NaN
1,223811159,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,...,07:00:00,25,0.0,00:00:00,15:10:00,ANT PROY TREN MAYA TRAMO 5,00:00:00,2022-11-30 17:05:56,NaT,NaN
2,223823654,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,...,00:20:00,20,5.0,01:50:05,02:02:00,ANTARA FASE II,01:53:55,2022-11-30 17:03:05,2022-11-30 17:28:12,NaN
3,223825052,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,...,07:00:00,25,0.0,02:56:47,15:10:00,ANT PROY TREN MAYA TRAMO 5,03:07:39,2022-11-30 18:22:42,2022-11-30 19:08:25,NaN
4,223851920,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,...,00:30:00,15,12.0,00:00:00,02:13:00,PILLM Pisos VESTA PARK,00:00:00,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396,223991571,D172,1715181,6.0,2022-12-03,11:36:24,CP FRACC CAMPANARIO (MCR),8.013550e+09,CR5074,L001,...,00:10:00,13,5.0,02:12:24,01:49:00,CP FRACC CAMPANARIO (MCR),02:12:40,2022-12-03 12:20:09,2022-12-03 13:48:48,NaN
71714,224549387,D103,1723608,6.0,2022-12-14,12:55:56,RINCONADA,8.013638e+09,CR3623,L003,...,00:45:00,15,32.0,03:08:18,02:46:00,RINCONADA,03:09:28,2022-12-14 13:16:01,2022-12-14 16:04:14,NaN
41055,224355876,D087,1352663,7.0,2022-12-10,06:33:00,VT02002 DEPARTAMENTOS MAGENTA,8.013604e+09,CR4746,L002,...,00:30:00,15,20.0,02:07:09,02:17:00,VT02002 DEPARTAMENTOS MAGENTA,02:08:03,2022-12-10 06:51:25,2022-12-10 08:40:09,NaN
42420,224360517,D216,1350925,7.0,2022-12-10,08:06:55,Aut Compostela-Las Varas PD,8.013623e+09,CR3897,L006,...,01:55:00,10,69.0,04:50:52,04:50:00,Aut Compostela-Las Varas PD,03:51:12,2022-12-10 10:15:46,2022-12-10 10:19:16,NaN


# Z1045_DSERV

In [21]:
DSERV = DSERV.drop(["Motivo_de_Pedido","Texto_breve_de_material","Producto_Comercial","Frente","Identif._externa_nota_de_entrega","Cantidad_entrada_de_mercancías","Nombre_Holding","Nombre_Jerarquía","Repr._Ventas","Factura","Posición","Creado_el","Hora"], axis=1)

In [22]:
DSERV["Fecha_inicio_real"] = DSERV["Fecha_inicio_real"].astype(str)
DSERV["Fecha_real_de_fin_de_la_orden"] = DSERV["Fecha_real_de_fin_de_la_orden"].astype(str)
DSERV["In_Site_Date"] = DSERV["In_Site_Date"].astype(str)
DSERV["Unld_Initial_Date"] = DSERV["Unld_Initial_Date"].astype(str)
DSERV["Unld_Final_Date"] = DSERV["Unld_Final_Date"].astype(str)
DSERV["In-Plant_Hour_of_Delivery"] = DSERV["In-Plant_Hour_of_Delivery"].astype(str)
DSERV["Fecha_de_carga"] = DSERV["Fecha_de_carga"].astype(str)
DSERV["Fecha_de_entrega"] = DSERV["Fecha_de_entrega"].astype(str)

In [23]:
DSERV = DSERV.rename({'In-Plant_Hour_of_Delivery': 'In_Plant_Hour_of_Delivery'}, axis=1)

In [24]:
DSERV["Fecha_inicio_real"] = DSERV.Fecha_inicio_real.astype(str).str.split(' ', expand=True)[0]
DSERV["Fecha_real_de_fin_de_la_orden"] = DSERV.Fecha_real_de_fin_de_la_orden.astype(str).str.split(' ', expand=True)[0]
DSERV["In_Site_Date"] = DSERV.In_Site_Date.astype(str).str.split(' ', expand=True)[0]
DSERV["Unld_Initial_Date"] = DSERV.Unld_Initial_Date.astype(str).str.split(' ', expand=True)[0]
DSERV["Unld_Final_Date"] = DSERV.Unld_Final_Date.astype(str).str.split(' ', expand=True)[0]
DSERV["In_Plant_Hour_of_Delivery"] = DSERV["In_Plant_Hour_of_Delivery"].astype(str).str.split(' ', expand=True)[0]
DSERV["Fecha_de_carga"] = DSERV.Fecha_de_carga.astype(str).str.split(' ', expand=True)[0]
DSERV["Fecha_de_entrega"] = DSERV.Fecha_de_entrega.astype(str).str.split(' ', expand=True)[0]

In [25]:
DSERV["Fecha_inicio_real"] = DSERV["Fecha_inicio_real"].astype(str) + " " + DSERV["Hora_inicio_real"].astype(str)
DSERV["Fecha_real_de_fin_de_la_orden"] = DSERV["Fecha_real_de_fin_de_la_orden"].astype(str) + " " + DSERV["Fin_real_(hora)"].astype(str)
DSERV["In_Site_Date"] = DSERV["In_Site_Date"].astype(str) + " " + DSERV["In_Site_Hr"].astype(str)
DSERV["Unld_Initial_Date"] = DSERV["Unld_Initial_Date"].astype(str) + " " + DSERV["Unld_Initial_Hr"].astype(str)
DSERV["Unld_Final_Date"] = DSERV["Unld_Final_Date"].astype(str) + " " + DSERV["Unld_Final_Hr"].astype(str)
DSERV["In_Plant_Hour_of_Delivery"] = DSERV["In_Plant_Hour_of_Delivery"].astype(str) + " " + DSERV["In-Plant_Hour_of_Delivery.1"].astype(str)
DSERV["Fecha_de_carga"] = DSERV["Fecha_de_carga"].astype(str) + " " + DSERV["Hora_de_carga"].astype(str)
DSERV["Fecha_de_entrega"] = DSERV["Fecha_de_entrega"].astype(str) + " " + DSERV["Hora_de_entrega"].astype(str)


In [26]:
DSERV["Fecha_inicio_real"] = pd.to_datetime(DSERV["Fecha_inicio_real"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["Fecha_real_de_fin_de_la_orden"] = pd.to_datetime(DSERV["Fecha_real_de_fin_de_la_orden"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["In_Site_Date"] = pd.to_datetime(DSERV["In_Site_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["Unld_Initial_Date"] = pd.to_datetime(DSERV["Unld_Initial_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["Unld_Final_Date"] = pd.to_datetime(DSERV["Unld_Final_Date"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["In_Plant_Hour_of_Delivery"] = pd.to_datetime(DSERV["In_Plant_Hour_of_Delivery"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["Fecha_de_carga"] = pd.to_datetime(DSERV["Fecha_de_carga"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
DSERV["Fecha_de_entrega"] = pd.to_datetime(DSERV["Fecha_de_entrega"], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [27]:
DSERV = DSERV.drop(["Hora_inicio_real","Fin_real_(hora)", "In_Site_Hr","Unld_Initial_Hr","Unld_Final_Hr","In-Plant_Hour_of_Delivery.1","Hora_de_carga","Hora_de_entrega"], axis=1)

In [28]:
# Filtrar los datos unicos para realizar las comparaciones 
Unicos = DSERV[DSERV.Entrega.duplicated() == True]
Unicos = Unicos.Entrega.unique()
Unicos = pd.DataFrame(Unicos)
Unicos = Unicos.rename({0: 'Entrega'}, axis=1)
Duplicados = Unicos.merge(DSERV, on='Entrega', how='left')

In [29]:
Duplicados["Ponderacion1"] = np.where(Duplicados["Fecha_inicio_real"] >= Duplicados["Fecha_real_de_fin_de_la_orden"], 0, 1)
Duplicados["Ponderacion2"] = np.where(Duplicados["Fecha_real_de_fin_de_la_orden"] >= Duplicados["In_Site_Date"], 0, 1)
Duplicados["Ponderacion3"] = np.where(Duplicados["In_Site_Date"] >= Duplicados["Unld_Initial_Date"], 0, 1)
Duplicados["Ponderacion4"] = np.where(Duplicados["Unld_Initial_Date"] >= Duplicados["Unld_Final_Date"], 0, 1)
Duplicados["Ponderacion5"] = np.where(Duplicados["Unld_Final_Date"] >= Duplicados["In_Plant_Hour_of_Delivery"], 0, 1)
Duplicados["Ponderacion6"] = np.where(Duplicados["In_Plant_Hour_of_Delivery"] >= Duplicados["Fecha_de_carga"], 0, 1)
Duplicados["Ponderacion7"] = np.where(Duplicados["Fecha_de_carga"] >= Duplicados["Fecha_de_entrega"], 0, 1)

In [30]:
Duplicados["Ponderacion"] = Duplicados["Ponderacion1"] + Duplicados["Ponderacion2"] + Duplicados["Ponderacion3"] + Duplicados["Ponderacion4"] + Duplicados["Ponderacion5"] + Duplicados["Ponderacion6"] + Duplicados["Ponderacion7"]
Duplicados = Duplicados.drop(["Ponderacion1","Ponderacion2","Ponderacion3","Ponderacion4","Ponderacion5","Ponderacion6","Ponderacion7"], axis=1)

In [31]:
Duplicados = Duplicados.sort_values('Ponderacion',ascending=False)
Duplicados = Duplicados[Duplicados.Entrega.duplicated() == False]
Duplicados = Duplicados.drop(["Ponderacion"], axis=1)

In [32]:
Data = DSERV.merge(Unicos, on='Entrega', how='left',indicator=True)
Data = Data[Data['_merge'] == "left_only"]
temp = Data["Entrega"]
Data = Data.drop(["_merge","Entrega"], axis=1)
Data.insert(0, 'Entrega', temp)

In [33]:
Full2 = pd.concat([Data, Duplicados])

In [34]:
Full2

,Entrega,In_Site_Date,Unld_Initial_Date,Unld_Final_Date,In_Plant_Hour_of_Delivery,Clase_de_documento,Orden_de_Venta,Fecha_de_entrega,Fecha_de_carga,Fecha_inicio_real,...,Nombre_Canal,Nombre_Subcanal,Nombre_Especialidad,Offline,Fabricación_en_Línea,Elemento_a_Colar,Nombre_Vicepresidencia,Fecha_de_documento,Pedido_cliente,Nombre_Vendedor
1,224033453,2022-12-05 12:06:42,2022-12-05 12:55:34,2022-12-05 13:31:54,NaT,Factura Especial RMS,8.013604e+09,2022-12-06 12:05:00,2022-12-06 11:23:00,2022-12-05 11:29:05,...,Construcción,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,CUNETA,Construcción,2022-12-05 00:00:00,8.013604e+09,JAVIER ALONSO PATINO MANZANERO
3,224037375,2022-12-05 15:22:15,2022-12-05 15:48:59,2022-12-05 15:51:58,2022-12-05 17:19:35,Normal RMS,8.013605e+09,2022-12-06 15:15:00,2022-12-06 13:47:00,2022-12-05 13:49:51,...,Construcción,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,BASE,Construcción,2022-12-05 00:00:00,8.013605e+09,MIGUEL ALEJANDRO SANCHEZ ZAVALA
8,224073976,2022-12-06 00:01:57,2022-12-06 00:01:57,2022-12-06 00:04:17,2022-12-06 01:16:13,Normal RMS,8.013601e+09,2022-12-06 00:12:00,2022-12-05 23:20:00,2022-12-05 23:18:08,...,Construcción,Comercial,Salud/Parques/Educación Privados,0.0,1.0,PISO,Construcción,2022-12-05 00:00:00,8.013601e+09,OMAR CARLOS GRAJEDA FAJARDO
11,224076241,2022-12-05 23:49:19,2022-12-06 00:20:53,2022-12-06 00:37:35,2022-12-06 00:55:41,Normal RMS,8.013607e+09,2022-12-06 00:07:00,2022-12-05 23:10:00,2022-12-05 23:10:29,...,Construcción,Comercial,Salud/Parques/Educación Privados,0.0,1.0,OTROS,Construcción,2022-12-05 00:00:00,8.013607e+09,DIANA MARIBEL CASTREJON CERRO
12,224076245,2022-12-06 00:09:47,2022-12-06 00:39:24,2022-12-06 00:37:30,2022-12-06 01:06:11,Normal RMS,8.013602e+09,2022-12-06 00:02:00,2022-12-05 23:05:00,2022-12-05 23:24:12,...,Construcción,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,PAVIMENTO,Construcción,2022-12-05 00:00:00,8.013602e+09,DIEGO VILLANUEVA ANDRADE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101726,224381501,2022-12-11 13:25:55,2022-12-11 13:25:55,2022-12-11 13:25:55,2022-12-11 13:25:55,Normal RMS,8.013630e+09,2022-12-11 11:36:00,2022-12-11 09:55:00,2022-12-11 09:59:22,...,Construcción,Infraestructura,Vialidades Urbanas,0.0,1.0,OTROS,Construcción,2022-12-11 00:00:00,8.013630e+09,JOSE MANUEL DURON MACIP
38106,224251387,2022-12-08 14:24:05,2022-12-08 14:24:05,2022-12-08 14:24:05,2022-12-08 14:24:05,Normal RMS,8.013601e+09,2022-12-08 13:37:00,2022-12-08 13:02:00,2022-12-08 13:03:07,...,Construcción,Vivienda,Vivienda Social/Residencial en Seri,0.0,1.0,LOSA,Construcción,2022-12-08 00:00:00,8.013601e+09,MOISES AVILA CARDENAS
106252,224485697,2022-12-13 17:03:39,2022-12-13 17:03:39,2022-12-13 17:03:39,2022-12-13 17:03:39,Factura Especial RMS,8.013639e+09,2022-12-13 15:00:00,2022-12-13 14:15:00,2022-12-13 14:17:36,...,Construcción,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,ZAPATA,Construcción,2022-12-13 00:00:00,8.013639e+09,JAVIER ALONSO PATINO MANZANERO
14968,224172678,2022-12-07 08:37:26,2022-12-07 08:37:26,2022-12-07 08:37:26,2022-12-07 08:37:26,Normal RMS,8.013596e+09,2022-12-07 08:05:00,2022-12-07 07:30:00,2022-12-07 07:30:29,...,Construcción,Vivienda,Vivienda Social/Residencial en Seri,0.0,1.0,ENCOFRADO,Construcción,2022-12-07 00:00:00,8.013596e+09,MOISES AVILA CARDENAS


# EMP

In [35]:
EMP = EMP.drop(EMP.columns[[0,2,3,9,14,15,16]], axis = 1)

# CALL

CALL = CALL.drop(CALL.columns[[0,1,10,11,27,28,29,31,32,33,34,35,36,37,38,44,45,46,47,49,50,52,56,57,58,59,68,69,70,72,73,74,75,76,77,78,79,80,81,83]], axis = 1)

In [59]:
ctx = snowflake.connector.connect(user='juan.yam@ext.cemex.com',
                                     account='hg45590', 
                                     region = 'East-US-2.azure',
                                     warehouse='PRD_LND_MRP_SAP', 
                                     database='PRD_TDS_MX_PLANOPER_CONCRETO',       
                                     schema ='EXOP',
                                     authenticator="externalbrowser",
                                     autocommit=True)   

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [68]:
cs = ctx.cursor()
cs.execute('CREATE OR REPLACE TABLE "DELIV" ("Centro" text, "Conductor" text, "Cantidad_entrega" float, "Fecha_inicio_real" text,"Hora_inicio_real" text, "Nombre_de_la_Obra" text, "Documento_de_Venta" float,"Placa_Vehículo_1" text, "Carga" text, "In-Plant_Hour_of_Delivery" text, "In-Plant_Hour_of_Delivery.1" text, "Estatus" text, "Obra" text, "Entrega" float, "Nombre_1" text, "Fecha_de_entrega" text, "In_Site_Date" text, "In_Site_Hr" text, "Unld_Initial_Date" text, "Unld_Initial_Hr" text, "Unld_Final_Date" text, "Unld_Final_Hr" text, "Date_of_Assigned_Delivery" text, "Hour_of_Assigned_Delivery" text, "Texto_breve_de_material" text, "Producto_Comercial" text, "Orden_de_Producción" float, "Fecha_de_documento" text, "Fecha_de_entrega.1" text, "Cliente" float, "Ticket" text, "Fecha_de_carga" text, "Hora_de_entrega" text, "Fecha_real_de_fin_de_la_orden" text, "Fin_real_(hora)" text, "Tiempo_de_trayecto" text, "T._Validación_mezcla_(mins)" text, "Distancia" text, "Ciclo_Real" text, "Ciclo_Teórico" text, "Nombre_del_Frente" text, "Cycle_Time" text, "Fecha_TJST" text, "Hora_TJST" text, "Fecha_TOPL" text, "Hora_TOPL" text, "Hora_de_carga" text, "Motivo_de_Pedido" text)')

In [69]:
#, "Producto_Comercial" text, "Orden_de_Producción" float, "Fecha_de_documento" date, "Fecha_de_entrega.1" date, "Cliente" float, "Ticket" text, "Fecha_de_carga" date, "Hora_de_entrega" text, "Fecha_real_de_fin_de_la_orden" date, "Fin_real_(hora)" text, "Tiempo_de_trayecto" text, "T._Validación_mezcla_(mins)" text, "Distancia" text, "Ciclo_Real" text, "Ciclo_Teórico" text, "Nombre_del_Frente" text, "Cycle_Time" text, "Fecha_TJST" date, "Hora_TJST" text, "Fecha_TOPL" date, "Hora_TOPL" text, "Hora_de_carga" text, "Motivo_de_Pedido" date

In [70]:
#cs.execute('DROP TABLE DELIV')

In [71]:
write_pandas(ctx,DELIV,table_name="DELIV")

(True,
 1,
 10,
 [('dcnisxbgwm/file0.txt', 'LOADED', 10, 10, 1, 0, None, None, None, None)])

In [72]:
cs.close()

True

In [57]:
DELIV

,Centro,Conductor,Cantidad_entrega,Fecha_inicio_real,Hora_inicio_real,Nombre_de_la_Obra,Documento_de_Venta,Placa_Vehículo_1,Carga,In-Plant_Hour_of_Delivery,...,Ciclo_Real,Ciclo_Teórico,Nombre_del_Frente,Cycle_Time,Fecha_TJST,Hora_TJST,Fecha_TOPL,Hora_TOPL,Hora_de_carga,Motivo_de_Pedido
0,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,2022-11-30,...,01:41:00,02:02:00,ANTARA FASE II,02:01:24,2022-11-30,15:31:40,2022-11-30,15:57:14,15:16:00,NaN
1,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,NaT,...,00:00:00,15:10:00,ANT PROY TREN MAYA TRAMO 5,00:00:00,2022-11-30,17:05:56,NaT,00:00:00,16:40:00,NaN
2,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,2022-11-30,...,01:50:05,02:02:00,ANTARA FASE II,01:53:55,2022-11-30,17:03:05,2022-11-30,17:28:12,17:00:00,NaN
3,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,2022-11-30,...,02:56:47,15:10:00,ANT PROY TREN MAYA TRAMO 5,03:07:39,2022-11-30,18:22:42,2022-11-30,19:08:25,17:00:00,NaN
4,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,NaT,...,00:00:00,02:13:00,PILLM Pisos VESTA PARK,00:00:00,NaT,00:00:00,NaT,00:00:00,23:11:00,NaN
5,D251,1724209,7.0,2022-11-30,23:29:17,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L023,2022-12-01,...,01:27:07,02:13:00,PILLM Pisos VESTA PARK,01:28:02,2022-11-30,23:37:27,2022-12-01,00:34:14,23:29:00,NaN
6,D251,1724545,7.0,2022-11-30,23:39:19,PILLM Pisos VESTA PARK,8.013569e+09,CR4538,L024,2022-12-01,...,02:00:53,02:13:00,PILLM Pisos VESTA PARK,02:01:52,2022-12-01,00:00:58,2022-12-01,00:44:05,23:39:00,NaN
7,D193,1724256,6.0,2022-11-30,23:08:56,CONCREA KOHLER (ERGP),8.013586e+09,CR5562,L004,2022-12-01,...,01:47:38,02:25:00,CONCREA KOHLER (ERGP),01:49:15,2022-11-30,23:35:06,2022-12-01,00:14:47,23:11:00,NaN
8,D175,1723979,7.0,2022-11-30,23:14:09,ENRIQUE DIAZ DE LEON (DVA),8.013558e+09,CR4608,L003,2022-12-01,...,01:49:24,02:20:00,ENRIQUE DIAZ DE LEON (DVA),01:52:49,2022-11-30,23:35:39,2022-12-01,00:37:20,23:13:00,NaN
9,D077,1722592,7.0,2022-11-30,23:12:45,Trolebus Chalco Santa Martha,8.013567e+09,CR5170,L007,2022-12-01,...,01:35:48,02:37:00,SECTOR 6,01:36:37,2022-11-30,23:28:35,2022-12-01,00:09:20,23:12:00,NaN


In [24]:
DELIV["Obra"] = DELIV["Obra"].astype(str)

In [49]:
DELIV["Fecha_inicio_real"] = DELIV["Fecha_inicio_real"].astype(str)
DELIV["In-Plant_Hour_of_Delivery"] = DELIV["In-Plant_Hour_of_Delivery"].astype(str)
DELIV["Fecha_de_entrega"] = DELIV["Fecha_de_entrega"].astype(str)
DELIV["In_Site_Date"] = DELIV["In_Site_Date"].astype(str)
DELIV["Unld_Initial_Date"] = DELIV["Unld_Initial_Date"].astype(str)
DELIV["Unld_Final_Date"] = DELIV["Unld_Final_Date"].astype(str)
DELIV["Date_of_Assigned_Delivery"] = DELIV["Date_of_Assigned_Delivery"].astype(str)
DELIV["Fecha_de_documento"] = DELIV["Fecha_de_documento"].astype(str)
DELIV["Fecha_de_entrega.1"] = DELIV["Fecha_de_entrega.1"].astype(str)
DELIV["Fecha_de_carga"] = DELIV["Fecha_de_carga"].astype(str)
DELIV["Fecha_real_de_fin_de_la_orden"] = DELIV["Fecha_real_de_fin_de_la_orden"].astype(str)
DELIV["Fecha_TJST"] = DELIV["Fecha_TJST"].astype(str)
DELIV["Fecha_TOPL"] = DELIV["Fecha_TOPL"].astype(str)
#DELIV["Fecha_inicio_real"] = DELIV["Fecha_inicio_real"].astype(str)

In [50]:
DELIV.dtypes

Centro                            object
Conductor                         object
Cantidad_entrega                 float64
Fecha_inicio_real                 object
Hora_inicio_real                  object
Nombre_de_la_Obra                 object
Documento_de_Venta               float64
Placa_Vehículo_1                  object
Carga                             object
In-Plant_Hour_of_Delivery         object
In-Plant_Hour_of_Delivery.1       object
Estatus                           object
Obra                              object
Entrega                            int64
Nombre_1                          object
Fecha_de_entrega                  object
In_Site_Date                      object
In_Site_Hr                        object
Unld_Initial_Date                 object
Unld_Initial_Hr                   object
Unld_Final_Date                   object
Unld_Final_Hr                     object
Date_of_Assigned_Delivery         object
Hour_of_Assigned_Delivery         object
Texto_breve_de_m

In [58]:
DELIV["In_Site_Date"]

0    2022-11-30
1           NaT
2    2022-11-30
3    2022-11-30
4           NaT
5    2022-11-30
6    2022-12-01
7    2022-12-01
8    2022-12-01
9    2022-11-30
Name: In_Site_Date, dtype: object

In [ ]:
pip uninstall pyarrow

In [ ]:
pip install pyarrow==8.0.0

In [ ]:
#cs.execute('CREATE TABLE "IK172" ("Linea_seleccionada" text, "Documento_medicion" text,	"Punto_de_medida" text, "Fecha" text, "Hora_medicion" text,	 "ValMed_Val_total_cont" text, "UnidadCaracteristica" text,	 "Creado_por" text,	 "Posicion_de_medida" text,	"Diferencia_val_cont" text,	"Equipo" text,	"Texto1" text,"Texto2" text)')


In [ ]:
cs.execute('CREATE [ OR REPLACE ] TABLE "DELIV" ("Centro" text, "Conductor" text, "Cantidad_entrega" float, "Fecha_inicio_real" date,"Hora_inicio_real" text, "Nombre_de_la_Obra" text, "Documento_de_Venta" text,"Placa_Vehículo_1" text, "Carga" text, "In-Plant_Hour_of_Delivery" date, "In-Plant_Hour_of_Delivery.1" text, "Estatus" text, "Obra" text, "Entrega" text, "Nombre_1" text, "Fecha_de_entrega" date, "In_Site_Date" date, "In_Site_Hr" text, "Unld_Initial_Date" date, "Unld_Initial_Hr" text, "Unld_Final_Date" date, "Unld_Final_Hr" text, "Date_of_Assigned_Delivery" date, "Hour_of_Assigned_Delivery" text, "Texto_breve_de_material" text, "Producto_Comercial" text, "Orden_de_Producción" text, "Fecha_de_documento" date, "Fecha_de_entrega.1" date, "Cliente" text, "Ticket" text, "Fecha_de_carga" date, "Hora_de_entrega" text, "Fecha_real_de_fin_de_la_orden" date, "Fin_real_(hora)" text, "Tiempo_de_trayecto" text, "T._Validación_mezcla_(mins)" text, "Distancia" text, "Ciclo_Real" text, "Ciclo_Teórico" text, "Nombre_del_Frente" text, "Cycle_Time" text, "Fecha_TJST" date, "Hora_TJST" text, "Fecha_TOPL" date, "Hora_TOPL" text, "Hora_de_carga" text, "Motivo_de_Pedido" date)')

In [ ]:
cs = ctx.cursor()
cs.execute("DROP TABLE IK172" )
cs.execute('CREATE TABLE "IK172" ("Linea_seleccionada" text, "Documento_medicion" text,	"Punto_de_medida" text, "Fecha" text, "Hora_medicion" text,	 "ValMed_Val_total_cont" text, "UnidadCaracteristica" text,	 "Creado_por" text,	 "Posicion_de_medida" text,	"Diferencia_val_cont" text,	"Equipo" text,	"Texto1" text,"Texto2" text)')
write_pandas(ctx,df,table_name="IK172")

In [37]:
DELIV2 = DELIV.drop(DELIV.columns[[25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]], axis = 1)

In [35]:
DELIV.head()

,Centro,Conductor,Cantidad_entrega,Fecha_inicio_real,Hora_inicio_real,Nombre_de_la_Obra,Documento_de_Venta,Placa_Vehículo_1,Carga,In-Plant_Hour_of_Delivery,...,Ciclo_Real,Ciclo_Teórico,Nombre_del_Frente,Cycle_Time,Fecha_TJST,Hora_TJST,Fecha_TOPL,Hora_TOPL,Hora_de_carga,Motivo_de_Pedido
0,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,2022-11-30,...,01:41:00,02:02:00,ANTARA FASE II,02:01:24,2022-11-30,15:31:40,2022-11-30,15:57:14,15:16:00,NaN
1,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,NaT,...,00:00:00,15:10:00,ANT PROY TREN MAYA TRAMO 5,00:00:00,2022-11-30,17:05:56,NaT,00:00:00,16:40:00,NaN
2,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,2022-11-30,...,01:50:05,02:02:00,ANTARA FASE II,01:53:55,2022-11-30,17:03:05,2022-11-30,17:28:12,17:00:00,NaN
3,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,2022-11-30,...,02:56:47,15:10:00,ANT PROY TREN MAYA TRAMO 5,03:07:39,2022-11-30,18:22:42,2022-11-30,19:08:25,17:00:00,NaN
4,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,NaT,...,00:00:00,02:13:00,PILLM Pisos VESTA PARK,00:00:00,NaT,00:00:00,NaT,00:00:00,23:11:00,NaN


In [39]:
DELIV2

,Centro,Conductor,Cantidad_entrega,Fecha_inicio_real,Hora_inicio_real,Nombre_de_la_Obra,Documento_de_Venta,Placa_Vehículo_1,Carga,In-Plant_Hour_of_Delivery,...,Fecha_de_entrega,In_Site_Date,In_Site_Hr,Unld_Initial_Date,Unld_Initial_Hr,Unld_Final_Date,Unld_Final_Hr,Date_of_Assigned_Delivery,Hour_of_Assigned_Delivery,Texto_breve_de_material
0,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,2022-11-30,...,2022-12-01,2022-11-30,15:52:20,2022-11-30,15:52:20,2022-11-30,15:52:43,2022-11-30,14:58:18,A-400-2-C-28-22-1-3-301
1,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,NaT,...,2022-12-01,NaT,00:00:00,NaT,00:00:00,NaT,00:00:00,2022-11-30,15:38:20,1-150-2-C-28-10-0-3-000
2,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,2022-11-30,...,2022-12-01,2022-11-30,17:14:25,2022-11-30,17:17:21,2022-11-30,17:17:43,2022-11-30,16:50:31,A-400-2-C-28-22-1-3-301
3,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,2022-11-30,...,2022-12-01,2022-11-30,18:44:51,2022-11-30,18:57:57,2022-11-30,19:05:49,2022-11-30,16:59:59,1-150-2-C-28-10-0-3-000
4,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,NaT,...,2022-12-01,NaT,00:00:00,NaT,00:00:00,NaT,00:00:00,2022-11-30,23:12:41,B-042-4-B-28-10-0-3-204
5,D251,1724209,7.0,2022-11-30,23:29:17,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L023,2022-12-01,...,2022-12-01,2022-11-30,23:58:35,2022-12-01,00:22:40,2022-11-30,23:59:05,2022-11-30,23:28:22,B-042-4-B-28-10-0-3-204
6,D251,1724545,7.0,2022-11-30,23:39:19,PILLM Pisos VESTA PARK,8.013569e+09,CR4538,L024,2022-12-01,...,2022-12-01,2022-12-01,00:21:57,2022-12-01,00:25:36,2022-12-01,00:26:19,2022-11-30,23:38:20,B-042-4-B-28-10-0-3-204
7,D193,1724256,6.0,2022-11-30,23:08:56,CONCREA KOHLER (ERGP),8.013586e+09,CR5562,L004,2022-12-01,...,2022-12-01,2022-12-01,00:08:17,2022-12-01,00:08:17,2022-12-01,00:12:38,2022-11-30,23:07:19,B-042-4-C-28-14-0-3-20K
8,D175,1723979,7.0,2022-11-30,23:14:09,ENRIQUE DIAZ DE LEON (DVA),8.013558e+09,CR4608,L003,2022-12-01,...,2022-12-01,2022-12-01,00:17:43,2022-12-01,00:24:53,2022-12-01,00:29:18,2022-11-30,23:10:44,P-045-4-B-14-10-0-3-36T
9,D077,1722592,7.0,2022-11-30,23:12:45,Trolebus Chalco Santa Martha,8.013567e+09,CR5170,L007,2022-12-01,...,2022-12-01,2022-11-30,23:53:22,2022-11-30,23:57:24,2022-12-01,00:03:38,2022-11-30,23:11:56,4-300-2-C-28-48-1-E-12W


In [206]:
#EMP

In [ ]:
EMP = EMP.drop(EMP.columns[[0,2,3,9,14,15,16]], axis = 1)

In [207]:
EMP

,Planta,Nombre_Planta,Pedido.,Nombre_cliente,Cliente,Nombre_Obra,Obra,Nombre_Frente,Frente,Producto_comercial,...,Bomba_asignada.1,Bomba_Específica,Mangueras,Cuadrilla,Tiempo_de_Instal.,Inicio_de_Bombeo,Fin_de_Bombeo,Tiempo_de_Inst._Bomba,Renegociado,Text
0,D330,MX-DG51 PD0441 DZITYA,8013273607-P-S,KELMAN DESARROLLOS S.A. DE C.V.,51024090,ARTHHOUSE RLA,67007161,ARTHHOUSE RLA,67007161,20000114,...,11003021.0,NaN,1,1,70,18:00:00,23:19:00,01:10:00,0,CBP// EXTRA 1 + 10 TRAMOS + 1ML
1,D288,MX-PD-110 JALAPA,8013287865,CONSTRUCCIONES COVASI XALAPA SA DE,50334091,ARCO SUR,67088570,ARCO SUR,67088570,20058717,...,NaN,NaN,0,0,0,00:00:00,00:00:00,00:00:00,0,NaN
2,D033,MX-PD0446 TUNELES GUAMUCHIL,8013290477-P,CRECEMOS LA BAHIA SA DE CV,51049394,VT02002 MARALMA,67083827,VT02002 MARALMA,67083827,20083215,...,11003937.0,NaN,0,0,20,19:40:00,00:15:00,00:20:00,0,CBP
3,E671,MX-PD0555 Periferico Mty III K,8013292799,CEMEX CONCRETOS,7100,PD0555 Periferico Mty III Km 96 al,E671,COCONAL MR PAV II,67089472,20079319,...,NaN,NaN,0,0,0,00:00:00,00:00:00,00:00:00,0,NaN
4,D132,MX-PD0208 SAHAGUN,8013290042-P,MW STEEL SA DE CV,50797994,PT CERO VERTIDO (MJ),67091166,PT CERO VERTIDO (MJ),67091166,20053672,...,11003668.0,NaN,0,0,30,13:10:00,21:30:00,00:30:00,0,BPLU\nok carlos bplu367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438,D036,MX-PD0532 ALAMAR 2000,8013283194-P,RUBA DESARROLLOS SA DE CV,50019204,"CIRCINI, CINCEL, CETUS, CENTAURO Y",67087620,"CIRCINI, CINCEL, CETUS, CENTAURO Y",67087620,20000114,...,11003896.0,NaN,0,0,20,14:30:00,17:35:00,00:20:00,0,CBP
3439,D196,MX-PD-411 ZUAZUA,8013276343,PROYECTOS URBANOS ECOLOGICOS,51037473,PASEOS DEL ROBLE,67062528,PASEOS DEL ROBLE,67062528,20000047,...,NaN,NaN,0,0,0,00:00:00,00:00:00,00:00:00,0,NaN
3440,D211,MX-PD-149 GDL. MATRIZ,8013288451-P,CARDOLIV DESARROLLOS SA DE CV,51023741,VT02402-TEPALCATEPEC(DSC),67006130,VT02402-TEPALCATEPEC(DSC),67006130,20000103,...,11003932.0,NaN,0,0,20,14:44:00,21:26:00,00:20:00,2,CBP
3441,D033,MX-PD0446 TUNELES GUAMUCHIL,8013270239-P,INGENIERIA MEXICANA,50732962,VT02302 ALDEA HORTUS FASE I,66918823,COTO 13 BUGAMBILIA,67070812,20000114,...,11003667.0,NaN,0,0,20,19:20:00,20:10:00,00:20:00,0,CBP


In [ ]:
#CALL

In [ ]:
CALL = CALL.drop(CALL.columns[[0,1,10,11,27,28,29,31,32,33,34,35,36,37,38,44,45,46,47,49,50,52,56,57,58,59,68,69,70,72,73,74,75,76,77,78,79,80,81,83]], axis = 1)

# Procesamiento de los datos de las bases

In [53]:
import pandas as pd
import numpy as np
from datetime import datetime

In [24]:
# FUNCIONES
def Columnames(database, namebase):
    deletecol = 0
    a = 0
    a = database.columns
    a = list(a)
    deletecol = [lista for lista in a if "Unnamed" in lista]
    if(deletecol != 0):
        database = database.drop(columns = deletecol)
    
    #database = database.loc[0:9,:]
    database= database.dropna(how='all')
    new_names = []
    for i in database.columns.values:
        result = i.replace(' ','_')
        result = result + namebase
        new_names.append(result)
    database.columns = new_names
    return database

In [11]:
df = pd.read_excel("DELIV.xlsx", sheet_name='Sheet1')

In [12]:
df2 = pd.read_excel("DSERV.xlsx", sheet_name='Sheet1')

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

df = pd.read_excel("DELIV.xlsx", sheet_name='Sheet1')
df2 = pd.read_excel("DSERV.xlsx", sheet_name='Sheet1')
df3 = pd.read_excel("EMP.xlsx", sheet_name='Sheet1')
df4 = pd.read_excel("CALL.xlsx", sheet_name='Sheet1')

In [145]:
DELIV = df
DSERV = df2
#EMP = df3
#CALL = df4

In [146]:
DELIV = Columnames(DELIV, "_DELIV")
DSERV = Columnames(DSERV, "_DSERV")
#EMP = Columnames(EMP, "_EMP")
#CALL = Columnames(CALL, "_CALL")

# Eliminacion de duplicados

In [147]:
#Cambiar el tipo de datos a las columnas DELIV

DELIV["Fecha_inicio_real_DELIV"] = DELIV["Fecha_inicio_real_DELIV"].astype(str)
DELIV["In-Plant_Hour_of_Delivery_DELIV"] = DELIV["In-Plant_Hour_of_Delivery_DELIV"].astype(str)
DELIV["Fecha_de_entrega_DELIV"] = DELIV["Fecha_de_entrega_DELIV"].astype(str)
DELIV["In_Site_Date_DELIV"] = DELIV["In_Site_Date_DELIV"].astype(str)
DELIV["Unld_Initial_Date_DELIV"] = DELIV["Unld_Initial_Date_DELIV"].astype(str)
DELIV["Unld_Final_Date_DELIV"] = DELIV["Unld_Final_Date_DELIV"].astype(str)
DELIV["Date_of_Assigned_Delivery_DELIV"] = DELIV["Date_of_Assigned_Delivery_DELIV"].astype(str)
DELIV["Fecha_de_documento_DELIV"] = DELIV["Fecha_de_documento_DELIV"].astype(str)
DELIV["Fecha_de_entrega.1_DELIV"] = DELIV["Fecha_de_entrega.1_DELIV"].astype(str)
DELIV["Fecha_de_carga_DELIV"] = DELIV["Fecha_de_carga_DELIV"].astype(str)
DELIV["Fecha_real_de_fin_de_la_orden_DELIV"] = DELIV["Fecha_real_de_fin_de_la_orden_DELIV"].astype(str)
DELIV["Fecha_TJST_DELIV"] = DELIV["Fecha_TJST_DELIV"].astype(str)
DELIV["Fecha_TOPL_DELIV"] = DELIV["Fecha_TOPL_DELIV"].astype(str)

In [148]:
Data = DELIV.merge(DSERV.rename({'Entrega_DELIV': 'Entrega_DSERV'}, axis=1),
               left_on='Entrega_DELIV', right_on='Entrega_DSERV', how='left')

In [149]:
Data = Data[Data['Entrega_DSERV'].notna()]
#Data = Data.dropna()

In [ ]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = Data.Fecha_real_de_fin_de_la_orden_DSERV.astype(str).str.split(' ', expand=True)[0]
Data["Fecha_inicio_real_DSERV"] = Data.Fecha_inicio_real_DSERV.astype(str).str.split(' ', expand=True)[0]
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = Data["Fecha_real_de_fin_de_la_orden_DSERV"].astype(str) + " " + Data["Fin_real_(hora)_DSERV"].astype(str)
Data["Fecha_inicio_real_DSERV"] = Data["Fecha_inicio_real_DSERV"].astype(str) + " " + Data["Hora_inicio_real_DSERV"].astype(str)
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = pd.to_datetime(Data["Fecha_real_de_fin_de_la_orden_DSERV"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
Data["Fecha_inicio_real_DSERV"] = pd.to_datetime(Data["Fecha_inicio_real_DSERV"], errors='coerce', format='%Y-%m-%d %H:%M:%S')


In [150]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = Data.Fecha_real_de_fin_de_la_orden_DSERV.astype(str).str.split(' ', expand=True)[0]
Data["Fecha_inicio_real_DSERV"] = Data.Fecha_inicio_real_DSERV.astype(str).str.split(' ', expand=True)[0]

In [151]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = Data["Fecha_real_de_fin_de_la_orden_DSERV"].astype(str) + " " + Data["Fin_real_(hora)_DSERV"].astype(str)
Data["Fecha_inicio_real_DSERV"] = Data["Fecha_inicio_real_DSERV"].astype(str) + " " + Data["Hora_inicio_real_DSERV"].astype(str)

In [152]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = pd.to_datetime(Data["Fecha_real_de_fin_de_la_orden_DSERV"], errors='coerce', format='%Y-%m-%d %H:%M:%S')
Data["Fecha_inicio_real_DSERV"] = pd.to_datetime(Data["Fecha_inicio_real_DSERV"], errors='coerce', format='%Y-%m-%d %H:%M:%S')

In [153]:
Data["Ponderacion"] = np.where(Data["Fecha_real_de_fin_de_la_orden_DSERV"] <= Data["Fecha_inicio_real_DSERV"], '0', '1')

In [154]:
a = Data[Data["Ponderacion"] == "1"]
b = Data[Data["Ponderacion"] == "0"]

In [143]:
b["Fecha_real_de_fin_de_la_orden_DSERV"] - b["Fecha_inicio_real_DSERV"]

982        0 days 00:00:00
1171     -1 days +22:05:01
1631     -1 days +23:07:47
2150     -1 days +22:09:57
2351       0 days 00:00:00
                ...       
588839   -1 days +22:03:39
589238     0 days 00:00:00
590207     0 days 00:00:00
590343     0 days 00:00:00
590446   -1 days +23:05:18
Length: 679, dtype: timedelta64[ns]

In [140]:
a

,Centro_DELIV,Conductor_DELIV,Cantidad_entrega_DELIV,Fecha_inicio_real_DELIV,Hora_inicio_real_DELIV,Nombre_de_la_Obra_DELIV,Documento_de_Venta_DELIV,Placa_Vehículo_1_DELIV,Carga_DELIV,In-Plant_Hour_of_Delivery_DELIV,...,Nombre_Subcanal_DSERV,Nombre_Especialidad_DSERV,Offline_DSERV,Fabricación_en_Línea_DSERV,Elemento_a_Colar_DSERV,Nombre_Vicepresidencia_DSERV,Fecha_de_documento_DSERV,Pedido_cliente_DSERV,Nombre_Vendedor_DSERV,Ponderacion
0,D105,159956,6.0,2022-11-30,15:18:42,ANTARA FASE II,8.013570e+09,CR4362,L006,2022-11-30,...,Comercial,Plazas/Locales/C. Comerciales/Estac,0.0,1.0,NaN,Construcción,2022-11-30 00:00:00,8.013570e+09,OMAR MEDINA CALLEJAS,1
1,D678,1724160,7.0,2022-11-30,16:39:41,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR4826,L001,NaT,...,Infraestructura,Ferrocarriles,0.0,1.0,LOSA DE CIMENTACION,Industrial y Transformación,2022-11-30 00:00:00,8.013577e+09,POR DEFINIR ZONA SURESTE,1
2,D105,1706888,3.0,2022-11-30,16:54:21,ANTARA FASE II,8.013570e+09,CR4343,L008,2022-11-30,...,Comercial,Plazas/Locales/C. Comerciales/Estac,0.0,1.0,NaN,Construcción,2022-11-30 00:00:00,8.013570e+09,OMAR MEDINA CALLEJAS,1
3,D678,1709505,7.0,2022-11-30,18:00:44,ANTEPROYECTOS PAVIMENTOS,8.013577e+09,CR5156,L002,2022-11-30,...,Infraestructura,Ferrocarriles,0.0,1.0,LOSA DE CIMENTACION,Industrial y Transformación,2022-11-30 00:00:00,8.013577e+09,POR DEFINIR ZONA SURESTE,1
4,D251,1724209,7.0,2022-11-30,23:13:21,PILLM Pisos VESTA PARK,8.013569e+09,CR4458,L022,NaT,...,Comercial,Salud/Parques/Educación Privados,0.0,1.0,PISO,Construcción,2022-11-30 00:00:00,8.013569e+09,OMAR CARLOS GRAJEDA FAJARDO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590575,D001,1701547,7.0,2022-12-31,13:14:21,Hotel Dreams,8.013709e+09,CR3920,L003,2022-12-31,...,Comercial,Hotel / Motel / Resorts,0.0,1.0,LOSA ENTREPISO,Construcción,2022-12-31 00:00:00,8.013709e+09,DANTE DE JESUS PINEDA ROJAS,1
590576,D046,184760,3.5,2022-12-31,13:16:06,CP-I CARIBE (HRU),8.013704e+09,CR3190,L001,NaT,...,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,NaN,Construcción,2022-12-31 00:00:00,8.013704e+09,JUAN CARLOS JURADO LOBATO,1
590577,D001,1721221,7.0,2022-12-31,13:25:30,Hotel Dreams,8.013709e+09,CR2822,L004,2022-12-31,...,Comercial,Hotel / Motel / Resorts,0.0,1.0,LOSA ENTREPISO,Construcción,2022-12-31 00:00:00,8.013709e+09,DANTE DE JESUS PINEDA ROJAS,1
590578,D001,1725852,7.0,2022-12-31,14:34:41,Hotel Dreams,8.013709e+09,CR3849,L005,2022-12-31,...,Comercial,Hotel / Motel / Resorts,0.0,1.0,LOSA ENTREPISO,Construcción,2022-12-31 00:00:00,8.013709e+09,DANTE DE JESUS PINEDA ROJAS,1


In [141]:
b

,Centro_DELIV,Conductor_DELIV,Cantidad_entrega_DELIV,Fecha_inicio_real_DELIV,Hora_inicio_real_DELIV,Nombre_de_la_Obra_DELIV,Documento_de_Venta_DELIV,Placa_Vehículo_1_DELIV,Carga_DELIV,In-Plant_Hour_of_Delivery_DELIV,...,Nombre_Subcanal_DSERV,Nombre_Especialidad_DSERV,Offline_DSERV,Fabricación_en_Línea_DSERV,Elemento_a_Colar_DSERV,Nombre_Vicepresidencia_DSERV,Fecha_de_documento_DSERV,Pedido_cliente_DSERV,Nombre_Vendedor_DSERV,Ponderacion
982,D219,1709005,7.0,2022-12-01,21:31:14,BODEGA RINOL GUZMAN (DVMV),8.013557e+09,CR3907,L016,2022-12-01,...,Comercial,Salud/Parques/Educación Privados,0.0,1.0,PISO,Construcción,2022-12-01 00:00:00,8.013557e+09,DAYNA VERONICA MARTINEZ VALDEZ,0
1171,D366,1702795,4.0,2022-12-01,08:43:03,VT02602 CPI PACHECO (CAAM),8.013572e+09,CR5379,L003,2022-12-01,...,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,LOSA,Construcción,2022-12-01 00:00:00,8.013572e+09,CHRISTIAN ADRIAN ALVAREZ MORENO,0
1631,D045,1723939,5.5,2022-12-01,09:40:28,VT02402-Topete(FAV),8.013582e+09,CR3770,L001,2022-12-01,...,Vivienda,Vivienda Particular (No Desarrollad,0.0,1.0,PISO,Construcción,2022-12-01 00:00:00,8.013582e+09,FREDD ALEXIS GIL VILLARINO,0
2150,D372,1722220,7.0,2022-12-01,10:44:04,CCC GONZALEZ ORTEGA (FACS),8.013584e+09,CR3134,L004,NaT,...,Infraestructura,Energía,0.0,1.0,OTROS,Infraestructura y Gobierno,2022-12-01 00:00:00,8.013584e+09,FRANCISCO ABRAHAM CONTRERAS SO,0
2351,D173,1724205,5.0,2022-12-01,11:07:30,MAURO SANT ANGELO E1 15VIV (GM,8.013573e+09,CR3573,L001,NaT,...,Vivienda,Vivienda Social/Residencial en Seri,0.0,1.0,BANQUETA,Construcción,2022-12-01 00:00:00,8.013573e+09,GLORIA MARTHA ROJAS RIOS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588839,D013,1434408,7.0,2022-12-30,13:40:04,AEROPUERTO AMPL PLATAFORMA VOL,8.013683e+09,CR5031,L001,2022-12-30,...,Infraestructura,Aeropuertos,0.0,1.0,PAVIMENTO,Construcción,2022-12-30 00:00:00,8.013683e+09,LUIS HUMBERTO CARDENAS BAHENA,0
589238,D003,1724260,7.0,2022-12-30,13:39:50,CANCHAS DEPORTIVAS,8.013693e+09,CR3730,L001,2022-12-30,...,Comercial,Plazas/Locales/C. Comerciales/Estac,0.0,1.0,PISO,Construcción,2022-12-30 00:00:00,8.013693e+09,DIANA GABRIELA TERAN BOJORQUEZ,0
590207,D194,1721001,7.0,2022-12-31,08:16:44,VT02702 PLAZA COMERCIAL *EDB*,8.013705e+09,CR2852,L002,2022-12-31,...,Infraestructura,"Plazoletas, parques y jardines públ",0.0,1.0,NaN,Construcción,2022-12-31 00:00:00,8.013705e+09,ERIKA DAVILA BEAZ,0
590343,D083,1429609,7.5,2022-12-31,09:42:41,LA MEXICANA,8.013691e+09,CR4295,L001,2022-12-31,...,Vivienda,Vivienda Social/Residencial en Seri,0.0,1.0,LOSA DE CIMENTACION,Construcción,2022-12-31 00:00:00,8.013691e+09,MARCO ANTONIO GONZALEZ SORIANO,0


In [125]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"]

0         2022-11-30 15:24:34
1         2022-11-30 16:49:01
2         2022-11-30 16:58:01
3         2022-11-30 18:10:29
4         2022-11-30 23:20:28
                 ...         
590575    2022-12-31 13:20:54
590576    2022-12-31 13:20:35
590577    2022-12-31 13:31:25
590578    2022-12-31 14:40:00
590579    2022-12-31 06:08:31
Name: Fecha_real_de_fin_de_la_orden_DSERV, Length: 590553, dtype: object

In [126]:
Data["Fecha_real_de_fin_de_la_orden_DSERV"] = pd.to_datetime(Data["Fecha_real_de_fin_de_la_orden_DSERV"], errors='coerce', format='%Y-%m-%d %H:%M:%S')

0        2022-11-30 15:24:34
1        2022-11-30 16:49:01
2        2022-11-30 16:58:01
3        2022-11-30 18:10:29
4        2022-11-30 23:20:28
                 ...        
590575   2022-12-31 13:20:54
590576   2022-12-31 13:20:35
590577   2022-12-31 13:31:25
590578   2022-12-31 14:40:00
590579   2022-12-31 06:08:31
Name: Fecha_real_de_fin_de_la_orden_DSERV, Length: 590553, dtype: datetime64[ns]

In [ ]:
#convertir la columna de fecha y hora a solo la fecha 
df [' hora '] = pd. to_datetime (df ​​[' hora ']). dt . fecha

In [ ]:
DELIV.columns

In [ ]:
EMP = EMP.drop(EMP.columns[[0,2,3,9,14,15,16]], axis = 1)
CALL = CALL.drop(CALL.columns[[0,1,10,11,27,28,29,31,32,33,34,35,36,37,38,44,45,46,47,49,50,52,56,57,58,59,68,69,70,72,73,74,75,76,77,78,79,80,81,83]], axis = 1)
DSERV = DSERV.drop(["Motivo_de_Pedido_DSERV","Texto_breve_de_material_DSERV","Producto_Comercial_DSERV","Frente_DSERV","Identif._externa_nota_de_entrega_DSERV","Cantidad_entrada_de_mercancías_DSERV","Nombre_Holding_DSERV","Nombre_Jerarquía_DSERV","Repr._Ventas_DSERV","Factura_DSERV","Posición_DSERV","Creado_el_DSERV","Hora_DSERV"], axis=1)

In [ ]:
# Separar los dartos apartir de caracteres "-"
pedido = CALL["Pedido._CALL"].str.split('-', expand=True)
CALL["Pedido._CALL"] = pedido[0]

In [ ]:
Data = DELIV.merge(DSERV.rename({'Entrega_DELIV': 'Entrega_DSERV'}, axis=1),
               left_on='Entrega_DELIV', right_on='Entrega_DSERV', how='left')

In [ ]:
CALL["Pedido._CALL"] = CALL["Pedido._CALL"].astype('float64')

In [ ]:
Data = Data.merge(CALL.rename({'Orden_de_Venta_DSERV': 'Pedido._CALL'}, axis=1),
               left_on='Orden_de_Venta_DSERV', right_on='Pedido._CALL', how='left')

In [ ]:
Data = Data.merge(EMP.rename({'Entrega_DELIV': 'Entrega_EMP'}, axis=1),
               left_on='Entrega_DELIV', right_on='Entrega_EMP', how='left')

In [ ]:
Data = Data.drop(["Pedido._CALL","Entrega_EMP"], axis=1)

In [ ]:
len(DSERV['Entrega_DSERV'].unique())

In [ ]:
len(DELIV['Entrega_DELIV'].unique())

In [ ]:
len(Data)

In [ ]:
DSERV = DSERV.rename(columns={'Entrega_DSERV':'Entrega_DELIV'})


In [ ]:
a = DELIV.merge(DSERV, on='Entrega_DELIV', how='left')

In [ ]:
a = pd.merge(DELIV,DSERV, how='left')

In [ ]:
Data.columns

In [ ]:
a.head()

In [ ]:
len(DSERV)

In [ ]:
len(DELIV)

In [ ]:
DELIV.columns

In [ ]:
DSERV.columns

In [ ]:
len(DELIV)

In [ ]:
len(DSERV)

In [ ]:
len(CALL)

In [ ]:
len(EMP)

In [ ]:
len(Data)

In [ ]:
Data["Pedido._CALL"]

In [ ]:
DELIV.columns

In [ ]:
SERV['Fecha_inicio_real_SERV'] = SERV['Fecha_inicio_real_SERV'].astype(str) + " " + SERV['Hora_inicio_real_SERV'].astype(str)
SERV['In-Plant_Hour_of_Delivery_SERV'] = SERV['In-Plant_Hour_of_Delivery_SERV'].astype(str) + " " + SERV['In-Plant_Hour_of_Delivery.1_SERV'].astype(str)
SERV['In_Site_Date_SERV'] = SERV['In_Site_Date_SERV'].astype(str) + " " + SERV['In_Site_Hr_SERV'].astype(str)
SERV['Unld_Initial_Date_SERV'] = SERV['Unld_Initial_Date_SERV'].astype(str) + " " + SERV['Unld_Initial_Hr_SERV'].astype(str)
SERV['Unld_Final_Date_SERV'] = SERV['Unld_Final_Date_SERV'].astype(str) + " " + SERV['Unld_Final_Hr_SERV'].astype(str)
SERV['Date_of_Assigned_Delivery_SERV'] = SERV['Date_of_Assigned_Delivery_SERV'].astype(str) + " " + SERV['Hour_of_Assigned_Delivery_SERV'].astype(str)
SERV['Fecha_de_entrega.1_SERV'] = SERV['Fecha_de_entrega.1_SERV'].astype(str) + " " + SERV['Hora_de_entrega_SERV'].astype(str)
SERV['Fecha_real_de_fin_de_la_orden_SERV'] = SERV['Fecha_real_de_fin_de_la_orden_SERV'].astype(str) + " " + SERV['Fin_real_(hora)_SERV'].astype(str)
SERV['Fecha_TJST_SERV'] = SERV['Fecha_TJST_SERV'].astype(str) + " " + SERV['Hora_TJST_SERV'].astype(str)
SERV['Fecha_TOPL_SERV'] = SERV['Fecha_TOPL_SERV'].astype(str) + " " + SERV['Hora_TOPL_SERV'].astype(str)
SERV['Fecha_de_carga_SERV'] = SERV['Fecha_de_carga_SERV'].astype(str) + " " + SERV['Hora_de_carga_SERV'].astype(str)

In [ ]:
SERV = SERV.drop(['Hora_inicio_real_SERV','In-Plant_Hour_of_Delivery.1_SERV','In_Site_Hr_SERV','Unld_Initial_Hr_SERV','Unld_Final_Hr_SERV','Hour_of_Assigned_Delivery_SERV','Hora_de_entrega_SERV','Fin_real_(hora)_SERV','Hora_TJST_SERV','Hora_TOPL_SERV','Hora_de_carga_SERV'], axis=1)